# Multitask Regression/Classification Network State Reconstruction Demo

## Install libraries to runtime

If running in Google Colab, run the following setup cell once each time you open your project in order to install non-default package dependencies to the runtime. DO NOT RUN if you have already created a kernel with `qsttoolkit` as per the setup instructions in `README.md`.

In [1]:
!pip install numpy==1.26.4 pandas==2.2.2 scipy==1.13.1 matplotlib==3.9.2 seaborn==0.13.2 qutip==5.0.4 scikit-learn==1.5.1 tensorflow==2.10.0
!pip install -i https://test.pypi.org/simple/ qsttoolkit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 69.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.0/28.0 MB 54.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 57.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 578.0/578.0 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 41.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 45.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 62.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.7/438.7 kB 37.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 25.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.3/781.3 kB 41.3 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 3.5.0
    Uninstalling keras-3.5.0:
      Successfully uninstalled k

Looking in indexes: https://test.pypi.org/simple/


## Imports

All of QSTToolkit's features can be accessed by importing `qsttoolkit` and calling them directly. Most features sit within two main subpackages: `.data` and `.tomography`. In this example notebook, functions are called from their subpackage where relevant to show which features sit in each subpackage, and which lie in modules outside either:

In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import seaborn as sns
import matplotlib.pyplot as plt
import qutip as qt
import qsttoolkit as qst

## Data Preparation

We first create batches of optical quantum state vectors of seven different varieties. `qsttoolkit.data` provides classes to produce such batches, each state with randomised parameters within given limits, based on the [QuTiP](https://qutip.org/docs/4.0.2/index.html) framework:

In [3]:
dim = 32

fock_batch = qst.data.FockStates(n_states = 1000,
                                 N = dim,
                                 n_range = [0, dim])
coherent_batch = qst.data.CoherentStates(n_states = 1000,
                                         N = dim,
                                         alpha_magnitude_range = [1e-6, 3])
thermal_batch = qst.data.ThermalStates(n_states = 1000,
                                       N = dim,
                                       nbar_range = [0, dim])
num_batch = qst.data.NumStates(n_states = 1000,
                               N = dim,
                               types = ['17', 'M', 'P', 'P2', 'M2'])
binomial_batch = qst.data.BinomialStates(n_states = 1000,
                                         N = dim,
                                         S_range = [1, 10],
                                         mu_range = [0, 2])
cat_batch = qst.data.CatStates(n_states = 1000,
                               N = dim,
                               alpha_magnitude_range = [0, 10])
random_batch = qst.data.RandomStates(n_states = 1000,
                                     N = dim)
print('States generated')

/usr/local/lib/python3.10/dist-packages/qsttoolkit/data/state_batches.py:149: UserWarning: thermal states are currently initialised as density matrices. Calling the product of the .density_matrices() method is equivalent to simply calling .states() attribute. This may change in the future.
  warnings.warn("thermal states are currently initialised as density matrices. Calling the product of the .density_matrices() method is equivalent to simply calling .states() attribute. This may change in the future.")


States generated


/usr/local/lib/python3.10/dist-packages/qsttoolkit/data/state_batches.py:333: UserWarning: Random states are currently initialised as density matrices. Calling the product of the .density_matrices() method is equivalent to simply calling .states() attribute. This may change in the future.
  warnings.warn("Random states are currently initialised as density matrices. Calling the product of the .density_matrices() method is equivalent to simply calling .states() attribute. This may change in the future.")


Parameters for noise sources that are applied to the density matrix and Husimi-Q representation:

In [5]:
mixed_state_noise_noise_level = 0.3   # Mixed state noise coefficient
affine_theta = 30                     # Maximum affine transformation theta
affine_x = 0.1                        # Maximum affine transformation x shift
affine_y = 0.1                        # Maximum affine transformation y shift
additive_Gaussian_stddev = 0.01       # Additive Gaussian noise distribution standard deviation
salt_and_pepper_prob = 0.1            # Salt and pepper noise pixel cutout probability

We initialise the Husimi-Q phase space before creating measurement data:

In [6]:
latent_dim = 32
xgrid = np.linspace(-5, 5, latent_dim)
pgrid = np.linspace(-5, 5, latent_dim)

We then create density matrices and Husimi-Q measurement images, applying mixed state noise using `qsttoolkit.data.mixed_state_noise()` and measurement noise using `qsttoolkit.data.apply_measurement_noise()`:

In [7]:
fock_data = pd.DataFrame(columns=['label', 'density_matrix', 'Husimi-Q_function', 'state_parameter'])
coherent_data = pd.DataFrame(columns=['label', 'density_matrix', 'Husimi-Q_function', 'state_parameter'])
thermal_data = pd.DataFrame(columns=['label', 'density_matrix', 'Husimi-Q_function', 'state_parameter'])
num_data = pd.DataFrame(columns=['label', 'density_matrix', 'Husimi-Q_function', 'state_parameter'])
binomial_data = pd.DataFrame(columns=['label', 'density_matrix', 'Husimi-Q_function', 'state_parameter'])
cat_data = pd.DataFrame(columns=['label', 'density_matrix', 'Husimi-Q_function', 'state_parameter'])
random_data = pd.DataFrame(columns=['label', 'density_matrix', 'Husimi-Q_function', 'state_parameter'])
print("DataFrames initialised")

fock_densities = [qst.data.mixed_state_noise(dm, mixed_state_noise_noise_level) for dm in fock_batch.density_matrices()]
fock_data['label'] = ['fock']*len(fock_densities)
fock_data['density_matrix'] = [dm.full() for dm in fock_densities]
fock_data['Husimi-Q_function'] = [qst.data.apply_measurement_noise(qt.qfunc(dm, xgrid, pgrid), affine_theta, affine_x, affine_y, additive_Gaussian_stddev, salt_and_pepper_prob) for dm in fock_densities]
fock_data['state_parameter'] = fock_batch.params
print("Fock data generated")

coherent_densities = [qst.data.mixed_state_noise(dm, mixed_state_noise_noise_level) for dm in coherent_batch.density_matrices()]
coherent_data['label'] = ['coherent']*len(coherent_densities)
coherent_data['density_matrix'] = [dm.full() for dm in coherent_densities]
coherent_data['Husimi-Q_function'] = [qst.data.apply_measurement_noise(qt.qfunc(dm, xgrid, pgrid), affine_theta, affine_x, affine_y, additive_Gaussian_stddev, salt_and_pepper_prob) for dm in coherent_densities]
coherent_data['state_parameter'] = coherent_batch.params
print("Coherent data generated")

thermal_densities = [qst.data.mixed_state_noise(dm, mixed_state_noise_noise_level) for dm in thermal_batch.density_matrices()]
thermal_data['label'] = ['thermal']*len(thermal_densities)
thermal_data['density_matrix'] = [dm.full() for dm in thermal_densities]
thermal_data['Husimi-Q_function'] = [qst.data.apply_measurement_noise(qt.qfunc(dm, xgrid, pgrid), affine_theta, affine_x, affine_y, additive_Gaussian_stddev, salt_and_pepper_prob) for dm in thermal_densities]
thermal_data['state_parameter'] = thermal_batch.params
print("Thermal data generated")

num_densities = [qst.data.mixed_state_noise(dm, mixed_state_noise_noise_level) for dm in num_batch.density_matrices()]
num_data['label'] = ['num']*len(num_densities)
num_data['density_matrix'] = [dm.full() for dm in num_densities]
num_data['Husimi-Q_function'] = [qst.data.apply_measurement_noise(qt.qfunc(dm, xgrid, pgrid), affine_theta, affine_x, affine_y, additive_Gaussian_stddev, salt_and_pepper_prob) for dm in num_densities]
num_data['state_parameter'] = num_batch.params
print("Num data generated")

binomial_densities = [qst.data.mixed_state_noise(dm, mixed_state_noise_noise_level) for dm in binomial_batch.density_matrices()]
binomial_data['label'] = ['binomial']*len(binomial_densities)
binomial_data['density_matrix'] = [dm.full() for dm in binomial_densities]
binomial_data['Husimi-Q_function'] = [qst.data.apply_measurement_noise(qt.qfunc(dm, xgrid, pgrid), affine_theta, affine_x, affine_y, additive_Gaussian_stddev, salt_and_pepper_prob) for dm in binomial_densities]
binomial_data['state_parameter'] = binomial_batch.params
print("Binomial data generated")

cat_densities = [qst.data.mixed_state_noise(dm, mixed_state_noise_noise_level) for dm in cat_batch.density_matrices()]
cat_data['label'] = ['cat']*len(cat_densities)
cat_data['density_matrix'] = [dm.full() for dm in cat_densities]
cat_data['Husimi-Q_function'] = [qst.data.apply_measurement_noise(qt.qfunc(dm, xgrid, pgrid), affine_theta, affine_x, affine_y, additive_Gaussian_stddev, salt_and_pepper_prob) for dm in cat_densities]
cat_data['state_parameter'] = cat_batch.params
print("Cat data generated")

random_densities = [qst.data.mixed_state_noise(dm, mixed_state_noise_noise_level) for dm in random_batch.density_matrices()]
random_data['label'] = ['random']*len(random_densities)
random_data['density_matrix'] = [dm.full() for dm in random_densities]
random_data['Husimi-Q_function'] = [qst.data.apply_measurement_noise(qt.qfunc(dm, xgrid, pgrid), affine_theta, affine_x, affine_y, additive_Gaussian_stddev, salt_and_pepper_prob) for dm in random_densities]
random_data['state_parameter'] = random_batch.params
print("Random data generated")

data = pd.concat([fock_data, coherent_data, thermal_data, num_data, binomial_data, cat_data, random_data])
data = data.sample(frac=1).reset_index(drop=True)
print("Dataset generated")

DataFrames initialised
Fock data generated
Coherent data generated
Thermal data generated
Num data generated
Binomial data generated
Cat data generated
Random data generated
Dataset generated


Alternatively, QSTToolkit provides the `optical_state_dataset()` function which produces the above dataset in one function, which is intended to be used as a stamdardised dataset for training models:

In [8]:
data = qst.data.optical_state_dataset(dim=32, latent_dim=32)

AttributeError: module 'qsttoolkit.data' has no attribute 'optical_state_dataset'

## Modelling

Data preprocessing:

In [9]:
X = data['Husimi-Q_function']

y_class_raw = data['label']
label_encoder = LabelEncoder()
y_class_int = label_encoder.fit_transform(y_class_raw)

y_reg_real = data['state_parameter'].apply(lambda x: x.real)
y_reg_imag = data['state_parameter'].apply(lambda x: x.imag)
y_reg = np.array([y_reg_real, y_reg_imag]).T

X_train, X_test, y_train_class, y_test_class, y_train_reg, y_test_reg = train_test_split(X, y_class_int, y_reg, test_size=0.2, random_state=42)
y_train = {"classification_output": y_train_class, "regression_output": y_train_reg}    # Dictionaries to handle both regression and classification tasks
y_test = {"classification_output": y_test_class, "regression_output": y_test_reg}
latent_dim = X_train[0].shape[0]
X_train = np.array([x for x in X_train]).reshape(-1, latent_dim, latent_dim, 1)
X_test = np.array([x for x in X_test]).reshape(-1, latent_dim, latent_dim, 1)

We create an instance of the `qsttoolkit.tomography.MultitaskQuantumStateTomography` class, which handles the model architecture, compiling and training:

In [10]:
multitask_model = qst.tomography.MultitaskQuantumStateTomography(dim=32,
                                                                 X_train=X_train,
                                                                 X_test=X_test,
                                                                 y_train=y_train,
                                                                 y_test=y_test,
                                                                 label_encoder=label_encoder,
                                                                 early_stopping_patience=3,
                                                                 lr_scheduler_factor=0.5,
                                                                 lr_scheduler_patience=4)

Model compiling and training using the `.train` method:

In [14]:
multitask_model.train(optimizer='adam',
                      classification_loss='sparse_categorical_crossentropy', regression_loss='mse',
                      classification_loss_weight=1.0, regression_loss_weight=0.5,
                      classification_metric='accuracy', regression_metric='mae',
                      epochs=20, batch_size=32, validation_split=0.2)

Epoch 1/20
 4/70 [>.............................] - ETA: 4:39 - loss: 22.5510 - classification_output_loss: 1.9500 - regression_output_loss: 41.2020 - classification_output_accuracy: 0.3008 - regression_output_mae: 3.3949

KeyboardInterrupt: 

## Analysis and Evaluation

Inspection of accuracy and loss over training epochs:

In [ ]:
multitask_model.plot_training()

Model evaluation:

In [ ]:
multitask_model.evaluate_classification(include_confusion_matrix=True, include_classification_report=True)

In [ ]:
multitask_model.evaluate_regression()

Built-in `.infer` method to infer labels and parameters for given input measurements and invert the label encoding, returning arrays of predicted label strings and parameter values:

In [ ]:
print(multitask_model.infer(X_test))

`MultitaskQuantumStateTomography.model` can be treated as a `tensorflow.keras.Model` object, for example:

In [ ]:
multitask_model.model.summary()

## Reconstructing Density Matrices from Inferred Labels and Parameters

QSTToolkit provides the `StateReconstructor` class which takes lists of inferred state labels and parameters and reconstructs the corresponding density matrices using QuTiP functions:

In [ ]:
reconstructor = qst.tomography.StateReconstructor()

predicted_labels, predicted_state_parameters = multitask_model.infer(X_test)

reconstructor.add_data(true_labels = label_encoder.inverse_transform(y_test['classification_output']),
                       predicted_labels = predicted_labels,
                       true_state_parameters = y_test['regression_output'],
                       predicted_state_parameters = predicted_state_parameters,
                       true_states = [test.reshape(32,32) for test in X_test])

44/44 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step


In [ ]:
reconstructor.restrict_parameters(fock_n_range=[0,32], binomial_S_range=[1,10])

In [ ]:
reconstructor.predictions_df.head()

,true_label,predicted_label,true_state_parameter,predicted_state_parameter,restricted_predicted_state_parameter,true_state,reconstructed_state,reconstructed_density_matrix
0,coherent,coherent,0.090145,1.522285+0.001121j,1.522285+0.001121j,"[[0.0013154764018338094, 0.010621428468088896,...",NaN,NaN
1,random,thermal,0.000000,7.038320-0.031900j,7.038320-0.031900j,"[[0.005443629084445558, 0.00575912654923005, 0...",NaN,NaN
2,thermal,thermal,21.000000,5.793470-0.006691j,5.793470-0.006691j,"[[0.022705942004373536, 0.012967667180667334, ...",NaN,NaN
3,num,num,2.770352,4.359138-0.070977j,4.335775+0.000000j,"[[0.0023707020649585044, 0.015026923102303623,...",NaN,NaN
4,coherent,coherent,2.401666,0.679518+0.126949j,0.679518+0.126949j,"[[0.005460410918387504, 0.02960635653451574, 0...",NaN,NaN


In [ ]:
reconstructor.reconstruct(32)